In [1]:
# MIT License
#
# Copyright (c) 2021 Florian
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
import os
import numpy as np
from ctypes import cdll, POINTER, c_size_t, c_void_p

lib = cdll.LoadLibrary(os.path.abspath('convolve.so'))

In [2]:
def add_padding(matrix, padding):
    """Add padding to given matrix."""
    n, m = matrix.shape
    r, c = padding

    padded_matrix = np.zeros((n + r * 2, m + c * 2))
    padded_matrix[r: n + r, c: m + c] = matrix

    return padded_matrix

def cross_correlate_2d(matrix, kernel, mode='full', use_lib=True):
    """Convolve two 2-dimensional arrays."""
    nij = matrix.shape
    kij = kernel.shape
    if kij[0] % 2 == 0 or kij[1] % 2 == 0:
        raise ValueError('Kernel shape should be odd!')

    if mode == 'valid':
        pij = 0, 0
        mij = nij[0] - kij[0] + 1, nij[1] - kij[1] + 1
    elif mode == 'same':
        pij = (kij[0] - 1) // 2, (kij[1] - 1) // 2
        mij = nij[0], nij[1]
    elif mode == 'full':
        pij = kij[0] - 1, kij[1] - 1
        mij = nij[0] + kij[0] - 1, nij[1] + kij[1] - 1
    else:
        raise KeyError('Unknown mode!')

    if pij[0] > 0 or pij[1] > 0:
        matrix = add_padding(matrix, pij)

    npij = matrix.shape
    if npij[0] < kij[0] or npij[1] < kij[1]:
        raise ValueError(
            'Kernel can\'t be bigger than matrix in terms of shape!')

    if mij[0] <= 0 or mij[1] <= 0:
        raise ValueError(
            'Can\'t apply input parameters, one of resulting output dimension is non-positive.')

    matrix_out = np.zeros(mij)
    if use_lib:
        lib.convolve_2d(
            (c_size_t * 2)(*npij), c_void_p(matrix.ctypes.data),
            (c_size_t * 2)(*kij), c_void_p(kernel.ctypes.data),
            (c_size_t * 2)(*mij), c_void_p(matrix_out.ctypes.data)
        )
    else:
        for i in range(mij[0]):
            indices_x = list(range(i, i + kij[0]))

            for j in range(mij[1]):
                indices_y = list(range(j, j + kij[1]))

                submatrix = matrix[indices_x, :][:, indices_y]
                matrix_out[i][j] = np.sum(np.multiply(submatrix, kernel))

    return matrix_out

def convolve_2d(matrix, kernel, mode='full', use_lib=True):
    """Convolve two 2-dimensional arrays (flips kernel)."""
    kernel_flip = np.flipud(np.fliplr(kernel))
    kernel_flip = kernel_flip.ravel().reshape(kernel.shape)
    return cross_correlate_2d(matrix, kernel_flip, mode=mode, use_lib=use_lib)

In [3]:
from scipy.signal import correlate2d

n, k = 100, 61

a = cross_correlate_2d(np.ones((n, n)), np.ones((k ,k)), mode='valid')
b = correlate2d(np.ones((n, n)), np.ones((k, k)), mode='valid')
print((a == b).all())

a = cross_correlate_2d(np.ones((n, n)), np.ones((k, k)), mode='same')
b = correlate2d(np.ones((n, n)), np.ones((k, k)), mode='same')
print((a == b).all())

a = cross_correlate_2d(np.ones((n, n)), np.ones((k, k)), mode='full')
b = correlate2d(np.ones((n, n)), np.ones((k, k)), mode='full')
print((a == b).all())

True
True
True


In [4]:
import cv2
from scipy.signal import convolve2d

def process_image(image): 
  image = cv2.imread(image) 
  image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY) 
  return image

def gaussian_kernel(n=5, sigma=0.85):
    """\
    creates gaussian kernel with side length `l` and a sigma
    """
    ax = np.linspace(-(n - 1) / 2., (n - 1) / 2., n)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sigma))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

image = process_image('sample.jpg')

# gaussian_blur = 1/16 * np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
gaussian_blur = gaussian_kernel(21, sigma=10)

image_new = convolve_2d(image, gaussian_blur, mode='same')
# image_new = convolve2d(image, gaussian_blur, mode='same', use_lib=False)
# image_new = convolve2d(image, gaussian_blur, mode='same')

cv2.imwrite('sample_convolved.jpg', image_new)

True

In [5]:
%timeit convolve_2d(image, gaussian_blur, mode='same')
# %timeit convolve_2d(image, gaussian_blur, mode='same', use_lib=False)
%timeit convolve2d(image, gaussian_blur, mode='same')

34.1 ms ± 451 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
409 ms ± 6.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
